In [6]:
#!pip install pgmpy
#!pip install torch

In [7]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel

In [9]:
student_model = BayesianModel(
                               [('D', 'G'),
                                ('I', 'G'),
                                ('G', 'L'),
                                ('I', 'S')
                               ]
                             )

![贝叶斯网络结构](https://ss.csdn.net/p?https://mmbiz.qpic.cn/mmbiz_png/4lN1XOZshfcFcjk5sb3MwtKrY9BXVLvLpw8WHoibhmDLtibFkkCwTpSjELxOuRKk6OHn9bbhu25g3hdnKOoCwHNg/640?wx_fmt=png)

In [10]:
grade_cpd = TabularCPD( 
    variable='G', # 节点名称    
    variable_card=3, # 节点取值个数   
    values=[[0.3, 0.05, 0.9, 0.5], # 该节点的概率表    
    [0.4, 0.25, 0.08, 0.3], 
    [0.3, 0.7, 0.02, 0.2]], 
    evidence=['I', 'D'], # 该节点的依赖节点 
    evidence_card=[2, 2] # 依赖节点的取值个数    
)   
 
    
difficulty_cpd = TabularCPD(    
            variable='D',   
            variable_card=2,    
            values=[[0.6, 0.4]] 
)   
 
    
intel_cpd = TabularCPD( 
            variable='I',   
            variable_card=2,    
            values=[[0.7, 0.3]] 
)   
 
    
letter_cpd = TabularCPD(    
            variable='L',   
            variable_card=2,    
            values=[[0.1, 0.4, 0.99],   
            [0.9, 0.6, 0.01]],  
            evidence=['G'], 
            evidence_card=[3]   
)   
 
    
sat_cpd = TabularCPD(   
            variable='S',   
            variable_card=2,    
            values=[[0.95, 0.2],    
            [0.05, 0.8]],   
            evidence=['I'], 
            evidence_card=[2]   
)

In [11]:
student_model.add_cpds( 
    grade_cpd,  
    difficulty_cpd, 
    intel_cpd,  
    letter_cpd, 
    sat_cpd 
)

In [12]:
student_model.get_cpds()

[<TabularCPD representing P(G:3 | I:2, D:2) at 0x7fa16cabc7d0>,
 <TabularCPD representing P(D:2) at 0x7fa16cabc790>,
 <TabularCPD representing P(I:2) at 0x7fa16cabc810>,
 <TabularCPD representing P(L:2 | G:3) at 0x7fa16cabc850>,
 <TabularCPD representing P(S:2 | I:2) at 0x7fa16cabc890>]

In [13]:
student_model.get_independencies()

(D _|_ S, I)
(D _|_ I | S)
(D _|_ S | I)
(D _|_ L | G)
(D _|_ S | L, I)
(D _|_ L | S, G)
(D _|_ L, S | I, G)
(D _|_ S | L, I, G)
(D _|_ L | S, I, G)
(G _|_ S | I)
(G _|_ S | L, I)
(G _|_ S | D, I)
(G _|_ S | L, D, I)
(I _|_ D)
(I _|_ D | S)
(I _|_ L | G)
(I _|_ L | D, G)
(I _|_ L | S, G)
(I _|_ L | D, S, G)
(L _|_ S | I)
(L _|_ D, S, I | G)
(L _|_ S | D, I)
(L _|_ S, I | D, G)
(L _|_ D, I | S, G)
(L _|_ D, S | I, G)
(L _|_ I | D, S, G)
(L _|_ S | D, I, G)
(L _|_ D | S, I, G)
(S _|_ D)
(S _|_ L, D, G | I)
(S _|_ L | G)
(S _|_ D, G | L, I)
(S _|_ L, G | D, I)
(S _|_ L | D, G)
(S _|_ L, D | I, G)
(S _|_ G | L, D, I)
(S _|_ D | L, I, G)
(S _|_ L | D, I, G)

In [20]:
from pgmpy.inference import VariableElimination 
student_infer = VariableElimination(student_model)  
 
    
prob_G = student_infer.query(   
            variables=['G'],    
            evidence={'I': 1, 'D': 0})  
print(prob_G)

Eliminating: S: 100%|██████████| 2/2 [00:00<00:00, 436.43it/s]

+------+----------+
| G    |   phi(G) |
+======+==========+
| G(0) |   0.9000 |
+------+----------+
| G(1) |   0.0800 |
+------+----------+
| G(2) |   0.0200 |
+------+----------+


In [21]:
# 生成数据  
import numpy as np  
import pandas as pd 
 
    
raw_data = np.random.randint(low=0, high=2, size=(1000, 5)) 
data = pd.DataFrame(raw_data, columns=['D', 'I', 'G', 'L', 'S'])    

data.head()

,D,I,G,L,S
0,0,1,0,0,0
1,1,1,1,1,1
2,1,0,1,0,0
3,1,1,0,1,1
4,0,1,1,1,1


In [23]:
# 定义模型  
from pgmpy.models import BayesianModel  
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator  
 
    
model = BayesianModel([('D', 'G'), ('I', 'G'), ('I', 'S'), ('G', 'L')]) 
 
    
# 基于极大似然估计进行模型训练    
model.fit(data, estimator=MaximumLikelihoodEstimator)   
for cpd in model.get_cpds():    
    # 打印条件概率分布  
    print("CPD of {variable}:".format(variable=cpd.variable))   
    print(cpd,'\n')

CPD of D:
+------+-------+
| D(0) | 0.492 |
+------+-------+
| D(1) | 0.508 |
+------+-------+ 

CPD of G:
+------+---------------------+--------------------+---------------------+--------------------+
| D    | D(0)                | D(0)               | D(1)                | D(1)               |
+------+---------------------+--------------------+---------------------+--------------------+
| I    | I(0)                | I(1)               | I(0)                | I(1)               |
+------+---------------------+--------------------+---------------------+--------------------+
| G(0) | 0.5062240663900415  | 0.5059760956175299 | 0.49056603773584906 | 0.5102880658436214 |
+------+---------------------+--------------------+---------------------+--------------------+
| G(1) | 0.49377593360995853 | 0.4940239043824701 | 0.5094339622641509  | 0.4897119341563786 |
+------+---------------------+--------------------+---------------------+--------------------+ 

CPD of I:
+------+-------+
| I(0) | 